In [ ]:
import os
import itertools
from PIL import Image

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
train_data = "./Desktop/Dataset/Training/"
filepaths_train = []
labels_train = []

folders_train = os.listdir(train_data)
for folder in folders_train:
    folderpath = os.path.join(train_data, folder)
    filelist = os.listdir(folderpath)
    for file in filelist:
        fpath = os.path.join(folderpath, file)

        filepaths_train.append(fpath)
        labels_train.append(folder)

FSeries_train = pd.Series(filepaths_train, name="filepaths")
LSeries_train = pd.Series(labels_train, name="labels")

train_df = pd.concat([FSeries_train, LSeries_train], axis=1)


test_data = "./Desktop/Dataset/Testing/"
filepaths_test = []
labels_test = []

folders_test = os.listdir(test_data)
for folder in folders_test:
    folderpath = os.path.join(test_data, folder)
    filelist = os.listdir(folderpath)
    for file in filelist:
        fpath = os.path.join(folderpath, file)

        filepaths_test.append(fpath)
        labels_test.append(folder)


FSeries_test = pd.Series(filepaths_test, name="filepaths")
LSeries_test = pd.Series(labels_test, name="labels")

tst_df = pd.concat([FSeries_test, LSeries_test], axis=1)

In [ ]:
data_balance = train_df.labels.value_counts()
sns.set_palette("flare")


def custom_autopct(pct):
    total = sum(data_balance)
    val = int(round(pct * total / 100.0))
    return "{:.1f}%\n({:d})".format(pct, val)


fig, ax = plt.subplots()
ax.pie(data_balance, labels=data_balance.index, autopct=custom_autopct)

plt.title("Training Data Balance", weight="bold")
plt.axis("equal")
plt.show()

In [ ]:
valid_df, test_df = train_test_split(tst_df,  train_size= 0.5, shuffle= True, random_state=123)

In [ ]:
data_balance_train = len(train_df.value_counts())
data_balance_valid = len(valid_df.value_counts())
data_balance_test = len(test_df.value_counts())

data_balance = [data_balance_train, data_balance_valid, data_balance_test]

labels = ["Train Data", "Valid Data", "Test Data"]

plt.title("Train - Val - Test Balance", weight="bold")
plt.pie(data_balance, labels=labels, autopct=custom_autopct)
plt.show()

In [ ]:
batch_size = 32
epochs = 10

img_size = (224, 224)

tr_gen = ImageDataGenerator()
tst_gen = ImageDataGenerator()

train_gen = tr_gen.flow_from_dataframe(
    train_df,
    x_col="filepaths",
    y_col="labels",
    target_size=img_size,
    class_mode="categorical",
    color_mode="rgb",
    shuffle=True,
    batch_size=batch_size,
)

valid_gen = tst_gen.flow_from_dataframe(
    valid_df,
    x_col="filepaths",
    y_col="labels",
    target_size=img_size,
    class_mode="categorical",
    color_mode="rgb",
    shuffle=True,
    batch_size=batch_size,
)

test_gen = tst_gen.flow_from_dataframe(
    test_df,
    x_col="filepaths",
    y_col="labels",
    target_size=img_size,
    class_mode="categorical",
    color_mode="rgb",
    shuffle=False,
    batch_size=batch_size,
)

In [ ]:
model = Sequential([
        Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu", input_shape= (224,224,3)),
        MaxPooling2D((2, 2)),
        Dropout(0.25),  

        Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
        MaxPooling2D((2, 2)),
        Dropout(0.25),  

        Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
        MaxPooling2D((2, 2)),
        Dropout(0.25),  
        
        Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
        MaxPooling2D((2, 2)),
        Dropout(0.25),  

        Flatten(),

        Dense(256,activation = "relu"),
        Dense(64,activation = "relu"),
        Dense(4, activation = "softmax")
    ])

model.compile(Adamax(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_gen, epochs=epochs, verbose=1, validation_data=valid_gen, shuffle=False)

In [ ]:
alexnet=Sequential()
alexnet.add(Conv2D(96,kernel_size=(11,11),strides=(4,4),activation='relu',input_shape=(224,224,3)))
alexnet.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
alexnet.add(ZeroPadding2D((2,2)))
alexnet.add(Conv2D(256,kernel_size=(5,5),activation='relu',strides=(1,1)))
alexnet.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
alexnet.add(ZeroPadding2D((1,1)))
alexnet.add(Conv2D(384,kernel_size=(3,3),activation='relu',strides=(1,1)))
alexnet.add(ZeroPadding2D((1,1)))
alexnet.add(Conv2D(384,kernel_size=(3,3),activation='relu',strides=(1,1)))
alexnet.add(ZeroPadding2D((1,1)))
alexnet.add(Conv2D(256,kernel_size=(3,3),activation='relu',strides=(1,1)))
alexnet.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
alexnet.add(Flatten())
alexnet.add(Dense(4096,activation='relu'))
alexnet.add(Dense(4096,activation='relu'))
alexnet.add(Dense(4,activation='softmax'))

alexnet.compile(loss="categorical_crossentropy",optimizer="adam", metrics=["accuracy"])
    
history=alexnet.fit(train_gen, epochs=epochs, verbose=1, validation_data=valid_gen, shuffle=False)

In [ ]:
vgg16 = Sequential()
vgg16.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding='same',activation='relu'))
vgg16.add(Conv2D(64, kernel_size=(3,3),padding='same',activation='relu'))
vgg16.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
vgg16.add(Conv2D(128, kernel_size=(3,3),padding='same',activation='relu'))
vgg16.add(Conv2D(128, kernel_size=(3,3),padding='same',activation='relu'))
vgg16.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
vgg16.add(Conv2D(256, kernel_size=(3,3),padding='same',activation='relu'))
vgg16.add(Conv2D(256, kernel_size=(3,3),padding='same',activation='relu'))
vgg16.add(Conv2D(256, kernel_size=(3,3),padding='same',activation='relu'))
vgg16.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
vgg16.add(Conv2D(512, kernel_size=(3,3),padding='same',activation='relu'))
vgg16.add(Conv2D(512, kernel_size=(3,3),padding='same',activation='relu'))
vgg16.add(Conv2D(512, kernel_size=(3,3),padding='same',activation='relu'))
vgg16.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
vgg16.add(Conv2D(512, kernel_size=(3,3),padding='same',activation='relu'))
vgg16.add(Conv2D(512, kernel_size=(3,3),padding='same',activation='relu'))
vgg16.add(Conv2D(512, kernel_size=(3,3),padding='same',activation='relu'))
vgg16.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
vgg16.add(Flatten())
vgg16.add(Dense(4096,activation='relu'))
vgg16.add(Dense(4096,activation='relu'))
vgg16.add(Dense(4,activation='softmax'))

vgg16.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    
history=vgg16.fit(train_gen, epochs=epochs, verbose=1, validation_data=valid_gen, shuffle=False)

In [ ]:
# Define needed variables
tr_acc = history.history['accuracy']
tr_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
index_loss = np.argmin(val_loss)
val_lowest = val_loss[index_loss]
index_acc = np.argmax(val_acc)
acc_highest = val_acc[index_acc]

Epochs = [i+1 for i in range(len(tr_acc))]
loss_label = f'best epoch= {str(index_loss + 1)}'
acc_label = f'best epoch= {str(index_acc + 1)}'

# Plot training history
plt.figure(figsize= (20, 8))
plt.style.use('fivethirtyeight')

plt.subplot(1, 2, 1)
plt.plot(Epochs, tr_loss, 'r', label= 'Training loss')
plt.plot(Epochs, val_loss, 'g', label= 'Validation loss')
plt.scatter(index_loss + 1, val_lowest, s= 150, c= 'blue', label= loss_label)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(Epochs, tr_acc, 'r', label= 'Training Accuracy')
plt.plot(Epochs, val_acc, 'g', label= 'Validation Accuracy')
plt.scatter(index_acc + 1 , acc_highest, s= 150, c= 'blue', label= acc_label)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout
plt.show()

In [ ]:
train_score = model.evaluate(train_gen, verbose=1)
valid_score = model.evaluate(valid_gen, verbose=1)
test_score = model.evaluate(test_gen, verbose=1)

print(f"Train loss: {train_score[0]}")
print(f"Train Accuracy: {train_score[1]}")
print("-" * 20)
print( )
print(f"Validation Loss: {valid_score[0]}")
print(f"Validation Accuracy: {valid_score[1]}")
print("-" * 20)
print( )
print(f"Test Loss: {test_score[0]}")
print(f"Test Accuracy: {test_score[1]}")

In [ ]:
train_score = alexnet.evaluate(train_gen, verbose=1)
valid_score = alexnet.evaluate(valid_gen, verbose=1)
test_score = alexnet.evaluate(test_gen, verbose=1)

print(f"Train loss: {train_score[0]}")
print(f"Train Accuracy: {train_score[1]}")
print("-" * 20)
print( )
print(f"Validation Loss: {valid_score[0]}")
print(f"Validation Accuracy: {valid_score[1]}")
print("-" * 20)
print( )
print(f"Test Loss: {test_score[0]}")
print(f"Test Accuracy: {test_score[1]}")

In [ ]:
train_score = vgg16.evaluate(train_gen, verbose=1)
valid_score = vgg16.evaluate(valid_gen, verbose=1)
test_score = vgg16.evaluate(test_gen, verbose=1)

print(f"Train loss: {train_score[0]}")
print(f"Train Accuracy: {train_score[1]}")
print("-" * 20)
print( )
print(f"Validation Loss: {valid_score[0]}")
print(f"Validation Accuracy: {valid_score[1]}")
print("-" * 20)
print( )
print(f"Test Loss: {test_score[0]}")
print(f"Test Accuracy: {test_score[1]}")

In [ ]:
preds = model.predict(test_gen)  
y_pred = np.argmax(preds, axis=1) 

In [ ]:
preds = alexnet.predict(test_gen)  
y_pred = np.argmax(preds, axis=1) 

In [ ]:
preds = vgg16.predict(test_gen)  
y_pred = np.argmax(preds, axis=1) 

In [ ]:
g_dict = test_gen.class_indices
classes = list(g_dict.keys())

cm = confusion_matrix(test_gen.classes, y_pred)

plt.figure(figsize= (10, 10))
plt.imshow(cm, interpolation= 'nearest', cmap= plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()

tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation= 45)
plt.yticks(tick_marks, classes)

thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j], horizontalalignment= 'center', color= 'white' if cm[i, j] > thresh else 'black')
    
plt.tight_layout()
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

plt.show()

print(classification_report(test_gen.classes, y_pred, target_names=classes))